In [ ]:
import os
import re
import pandas as pd
import urllib
from nltk import SnowballStemmer
from gensim.models import Word2Vec
#import langdetect
import tika
import time
from tika import parser
import pickle

# Raw Articles pdf Data
Convert pdf to text using Tika apache server 

In [ ]:
# specify the target data folder
target_dir = 'C:/Thesis/Data/Journal Of management_2000_2020'

In [ ]:
# prep pdf extraction
pdf_files = []
wrd_files = []

for f in os.listdir(target_dir):
    if f.endswith(".pdf") | f.endswith(".PDF"):
        thispdf = os.path.join(target_dir, f)
        pdf_files = pdf_files + [thispdf]
    if f.endswith(".doc") | f.endswith(".docx") | f.endswith(".DOC") | f.endswith(".DOCX"):
        thiswrd = os.path.join(target_dir, f)
        wrd_files = wrd_files + [thiswrd]

In [ ]:
port = 4321 # port to use for Tika server (chosen arbitrarily)
server_str = 'http://localhost:' + str(port)
tika_path  = 'C:\Software\ tika-server-1.23.jar'
tika_run   = 'java -jar' + tika_path + ' --port ' + str(port)
print('Command to start Tika:')
print(tika_run)
print('---')
print('Number of pdf files: ' + str(len(pdf_files)))
print('Number of word files: ' + str(len(wrd_files)))

# PDF Metadata

In [ ]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

pdf_metadata = [parser.from_file(t, server_str, xmlContent=False)["metadata"] for t in (pdf_files + wrd_files)]

In [ ]:
# save data metadata
pickle.dump(pdf_metadata, open( "C:/Thesis/Data/save/journal_of_management/metadata/metadata_all.p", "wb" ) )

In [1]:
# load metadata
import pickle

pdf_metadata = pickle.load(open("C:/Thesis/Data/save/journal_of_management/metadata/metadata_all.p", "rb"))

In [3]:
#pdf_metadata

# pdf content

In [ ]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

#pdf_text = [parser.from_file(t, server_str, xmlContent=False)["content"] for t in (pdf_files + wrd_files)]

# Load pdf content

In [4]:
import pickle
#load the text data
pdf_data = pickle.load(open("C:/Thesis/Data/save/journal_of_management/save_alltext_jm.p", "rb"))


# Get title, author and creator from metadata

In [5]:
pdf_title = []
pdf_Author = []
pdf_subject = []
#pdf_creator =[]
#pdf_Keywords = []

for i in pdf_metadata:
    try:
        temp = i['title']
        auth_temp = i['Author']
        subj_temp = i['subject']
       #cre_temp = i['creator']
        #key_temp = i['Keywords']
        
        
        pdf_title.append(temp)
        pdf_Author.append(auth_temp)
        pdf_subject.append(subj_temp)
        #pdf_creator.append(cre_temp)
        #pdf_Keywords.append(key_temp)
    except:
        #print("An exception occurred" )
        pdf_title.append(temp)
        pdf_Author.append(auth_temp)
        pdf_subject.append(subj_temp)
        #pdf_creator.append(cre_temp)
        #pdf_Keywords.append(key_temp)

In [30]:
pdf_subject[1]

'Journal of Management 2005.31:680-699'

In [8]:
import re
# step-1 preprocess subject
pdf_sub = []

  # pre-processing for subject
for i in pdf_subject:
    # remove special character
    t = (re.sub(r'[?\-|.|:|!]','',i))
    # remove digits
    t = (re.sub("\d+", "", t))
    # remove end whitespace
    t = re.sub(r"\s+$","",t)
    # append to the list
    pdf_sub.append(t)

In [10]:
# step-2 remove subject
pdf_txt_sub = []

for i in pdf_data:
    line = i
       # for subject
    for m in pdf_sub:
        line = re.sub(m, '', line)

        
    pdf_txt_sub.append(line)
        

In [11]:
# save removed subject 
pickle.dump(pdf_txt_sub, open( "C:/Thesis/Data/save/journal_of_management/metadata/removed_subject/metadata_all.p", "wb" ) )


In [23]:
# step 3 pre-process author
# for author
pdf_au = []

for i in pdf_txt_sub:
    line = i
    for j in pdf_Author:
        if ',' in j:
            k = re.sub(' and ', ' ', j)
            l = k.split(",")
            for r in l:
                line = re.sub(r.strip(), '', line)

        else:
            k = re.sub(' and ', ',', j)
            l = k.split(",")
            for r in l:
                line = re.sub(r.strip(), '', line)

    pdf_au.append(line)

In [26]:
#pdf_au[1]

In [24]:
# save removed subject + author
pickle.dump(pdf_au, open( "C:/Thesis/Data/save/journal_of_management/metadata/removed_subject_author/metadata_all.p", "wb" ) )


In [27]:
# step-4 remove title

pdf_text_title = []
for i in pdf_au:
    line = i
    for j in pdf_title:
         line = re.sub(j, '', line)
    pdf_text_title.append(line)

In [28]:
pdf_text_title[1]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n10.1177/0149206305279054ARTICLE / October 2005Arendt et al. / CEO-Adviser Model\n\nA CEO-Adviser Model\nof Strategic Decision Making†\n\n*\nProfessional Programs in Business, University of Wisconsin–Green Bay,\n\n2420 Nicolet Drive, WH 460, Green Bay, WI 54311\n\n\nSchool of Business Administration, Management & Organizations Area,\n\nUniversity of Wisconsin–Milwaukee, P.O. Box 742, Milwaukee, WI 53201\n\n\nBelk College of Business Administration, University of North Carolina at Charlotte,\n\n9201 University City Blvd., Charlotte, NC 28223\n\nUpper echelons research has emphasized decision making either by individual CEOs or by teams\nof top managers. The authors introduce the CEO-Adviser model as an intermediate model of stra-\ntegic decision making. The CEO-Adviser model leads to new propositions that have not been\nexplored through the individual CEO or top management team models conce

In [31]:
# save removed subject + author
pickle.dump(pdf_text_title, open( "C:/Thesis/Data/save/journal_of_management/metadata/removed_subject_author_title/metadata_all.p", "wb" ) )


In [ ]:
# remove title, author 
# import re

# pdf_text = []
# pdf_sub = []

# # pre-processing for subject
# for i in pdf_subject:
#     # remove special character
#     t = (re.sub(r'[?\-|.|:|!]','',i))
#     # remove digits
#     t = (re.sub("\d+", "", t))
#     # remove end whitespace
#     t = re.sub(r"\s+$","",t)
#     # append to the list
#     pdf_sub.append(t)

# # filter out for author, title and subject
# for i in pdf_data:
#     line = i
#     # for author
#     for j in pdf_Author:
#         if ',' in j:
#             k = re.sub(' and ', ' ', j)
#             l = k.split(",")
#             for r in l:
#                 line = re.sub(r.strip(), '', line)
#         else:
#             k = re.sub(' and ', ',', j)
#             l = k.split(",")
#             for r in l:
#                 line = re.sub(r.strip(), '', line)
                
#     # for title       
#     for j in pdf_title:
#          line = re.sub(j, '', line)

#     # for subject
#     for m in pdf_sub:
#         line = re.sub(m, '', line)

        
#     pdf_text.append(line)
        

# 2 -  Convert pdf text to lower case

In [37]:
#Convert each string to lowercase
for i in range(len(pdf_text_title)):
    pdf_text_title[i] = pdf_text_title[i].lower()

In [38]:
# remove references section 
import re
pdf_ref = [re.sub(r"(?is)\nreferences\n.+", "", f) for f in pdf_text_title]

In [ ]:
# not required
def eliminate_method_result(method, discussion):
    
    import re    
    start = method
    end = discussion
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]

    return pdf_method_dis

In [ ]:
# not required
def eliminate_method_result_ref(method, dis, ref):
    
    import re

    start = method
    end = dis
    
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]
    
    start_ref = ref
    pattern_ref = start_ref +'(.|\n)*$'

    pdf_ref= [re.sub(pattern_ref, '', f) for f in pdf_method_dis]
    
    return pdf_ref

# 3 - Pre-processing
Regular expression is used to clean up the pre-processed data and saved as a Master data

In [42]:
# import re
# pdf_p = [re.sub("r[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?", " ", t)for t in pdf_ref]
# #remove hyphon only if it is at the end of the line.
# pdf_p = [re.sub(r'-\n(\w+ *)', r'\1\n', t) for t in pdf_p]
# # remove urls
# pdf_p = [re.sub( r'https?://[^\s<>"]+|www\.[^\s<>"]+', "", t)for t in pdf_p]
# #remove email address          
# pdf_p = [re.sub(r'\w+@\w+\.(?:com|net|org|live|edu)', "", t)for t in pdf_p]
# #remove text within ()
# pdf_p = [re.sub(r"\([^)]*\)", "", t)for t in pdf_p]
# # remove text with [] eg citation
# pdf_p = [re.sub(r"\[[^)]*\]", "", t)for t in pdf_p]
# #Remove numbers
# pdf_p = [re.sub('[0-9]+', '', f) for f in pdf_p]
# # remove single characters
# pdf_p = [re.sub(' [a-z] ' ,  ' ', f) for f in pdf_p]

# pdf_p = [re.sub(' [a-z]\.', '.', f) for f in pdf_p]

# pdf_p = [re.sub(' \.', '.', f) for f in pdf_p]

# pdf_p = [re.sub('[\n\r]+', ' ', f) for f in pdf_p]
# pdf_p = [re.sub('[!@/%#$]', ' ', f) for f in pdf_p]
# pdf_p = [re.sub(r"\s\([A-Z][a-z]+,\s[A-Z][a-z]?\.[^\)]*,\s\d{4}\)", "", f) for f in pdf_p]
# # save all periods, exclamation marks and question marks, but get rid of unnecessary characters
# pdf_p = [re.sub('[\.!?]+ ', 'XYZXYZ', t) for t in pdf_p]
# pdf_p = [re.sub('\W*-\W*', ' ', f) for f in pdf_p] # woordreparatie :)
# pdf_p = [re.sub('[\W_]+', ' ', f) for f in pdf_p]
# pdf_p = [re.sub('XYZXYZ', '. ', f) for f in pdf_p]
# #pdf_p[0]

In [47]:
pdf_p[0]

' . editorial february feldman i am delighted to introduce sage publications as the new publisher of the. as of january sage publications has taken over the production marketing and distribution of jom. as you can see there have been some style changes in the journal which we hope will increase its attractiveness to readers. sage publications will also begin some new marketing initiatives for jom and be working with the southern management association to streamline and improve our distribution system. there are no changes in submission guidelines or procedures for authors so from the perspective of prospective authors there will be no disruptions in the editorial review process at all. the year was great one for jom. we crossed the manuscript per year mark this past year and at the pace we are running so far this year we will almost certainly pass the manuscriptper year mark in. that will represent increase in submissions per year since took over as editor. at the same time we have kep

In [43]:
# Authors names are saved already (code is available at author name folder-get name list notebook)
#load authors names
import pickle
author_nam = pickle.load(open("C:/Thesis/Data/save/Master_Data/auth_ref_filter_num_char/auth_Journal_of_management.p", "rb"))

In [44]:
# GET REFERENCE LISTS OF STOPWORDS, I.E. WORDS THAT ARE ESSENTIALLY MEANINGLESS
# Get all the list of authors names

from nltk.corpus import stopwords

sw = stopwords.words('english') + author_nam + country_lower + cities_lower

#print(sw)

In [11]:
# Function to filter out the stopwords and authors names
def filterWords(msg, words):
    m = msg.split(' ')
    words_end = [w + '.' for w in words]
    filtered_words = list(filter(lambda word: word not in words, m))
    filtered_words = [(w if w not in words_end else '.') for w in filtered_words]
    result = ' '.join(filtered_words)
    return(result)

In [ ]:
# GET RID OF THE STOPWORDS IN TEXTS --running
data_proc_1 = [filterWords(f, sw) for f in pdf_p]

In [48]:
# save data
pickle.dump(data_proc_1, open( "C:/Thesis/Data/save/journal_of_management/afterfilter/data_proc_1.p", "wb" ) )

In [1]:
import pickle
#load the text data
data_proc_1 = pickle.load(open("C:/Thesis/Data/save/journal_of_management/afterfilter/data_proc_1.p", "rb"))

In [3]:
data_proc_1[122]

' antitakeover provisions shareholder value implications review contingency framework paper reviews literatures two competing theoretical views antitakeover provisions shareholder value relationship managerial entrenchment shareholder interests offers contingency framework reconcile views. framework based broader agency theory perspective organizational literatures power incorporates two key moderators corporate shareholder monitoring draws attention rather whether antitakeover provisions enhance erode shareholder value.  elsevier science . rights reserved. managerial responses active takeover market sparked considerable controversy. common strategic action taken managers even prior takeover bid adoption antitakeover provisions essentially involves company corporate charter operating policy makes takeover expensive. revival merger acquisition activity reactivated controversy additional adopting antitakeover provisions . moreover powerful shareholders pension funds increasingly challeng

In [7]:
import re
pdf_p= [re.sub(r"-\n(\n)*", "", t)for t in data_proc_1]

pdf_p= [re.sub(r"\n(\n)*", "\n", t)for t in pdf_p]

pdf_p= [re.sub(r"\n", " ", t)for t in pdf_p]
# remove text with [] eg citation
pdf_p = [re.sub(r"\[[^)]*\]", "", t)for t in pdf_p]
#Remove numbers
pdf_p = [re.sub('[0-9]+', '', f) for f in pdf_p]
#pdf_p = [re.sub("r[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?", " ", t)for t in pdf_p]
pdf_p = [re.sub('[!@/%“”‘:#©β<>+=δχ*&$]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('’s','', f) for f in pdf_p]
pdf_p = [re.sub('’re','', f) for f in pdf_p]
pdf_p = [re.sub('’t','', f) for f in pdf_p]
pdf_p = [re.sub('’ve','', f) for f in pdf_p]
pdf_p = [re.sub('’ll','', f) for f in pdf_p]
pdf_p = [re.sub('’m','', f) for f in pdf_p]

pdf_p = [re.sub('[:()-]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('[\.!?]+ ', 'XYZXYZ', t) for t in pdf_p]
# \w will match alphanumeric characters and underscores
# [^\w] will match anything that's not alphanumeric or underscore
pdf_p = [re.sub(r'[^\w]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('XYZXYZ', '.', f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]
# Replace multiple dots with one dot
pdf_p = [re.sub('\.\.+',".", f) for f in pdf_p]
pdf_p = [re.sub(r'\b\w{1,3}\b',"", f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]

pdf_p = [re.sub("^\s+","", f) for f in pdf_p]# remove the front space
pdf_p = [re.sub("\s+\Z","", f) for f in pdf_p]#remove the back space

pdf_p = [re.sub(' \.+', ' ', f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]
pdf_p = [re.sub('http', '', f) for f in pdf_p]
pdf_p = [re.sub('www', '', f) for f in pdf_p]
pdf_p = [re.sub('iweb', '', f) for f in pdf_p]
pdf_p = [re.sub('ttimmerman ', ' ', f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]


In [9]:
pdf_p[22]

'. january author reprints permissions sagepub journalspermissions meta analysis interactive additive relative effects cognitive ability motivation performance tested longstanding belief performance function interaction cognitive ability motivation.using data values obtained primary study authors input used meta analysis assess strength consistency multiplicative effects ability motivation performance.triangulation evidence based several types analyses revealed effects ability motivation performance additive rather multiplicative.additive effects ability motivation accounted explained variance performance whereas ability motivation interaction accounted explained variance addition interaction consistently reflect predicted form findings acknowledgments article accepted editorship previous version paper presented meetings academy management thank brian anonymous reviewers useful constructive feedback allowed improve manuscript substantially also thank guidance range restriction correcti

In [10]:
stopwords = ['a', 'about', 'above', 'across', 'after', 'afterwards']
stopwords += ['again', 'against', 'all', 'almost', 'alone', 'along']
stopwords += ['already', 'also', 'although', 'always', 'am', 'among']
stopwords += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stopwords += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stopwords += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stopwords += ['because', 'become', 'becomes', 'becoming', 'been']
stopwords += ['before', 'beforehand', 'behind', 'being', 'below']
stopwords += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stopwords += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stopwords += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stopwords += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stopwords += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stopwords += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stopwords += ['every', 'everyone', 'everything', 'everywhere', 'except']
stopwords += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stopwords += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stopwords += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stopwords += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stopwords += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stopwords += ['herself', 'him', 'himself', 'his', 'how', 'however']
stopwords += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stopwords += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stopwords += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stopwords += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stopwords += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stopwords += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stopwords += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stopwords += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stopwords += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stopwords += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stopwords += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stopwords += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stopwords += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stopwords += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stopwords += ['some', 'somehow', 'someone', 'something', 'sometime']
stopwords += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stopwords += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stopwords += ['then', 'thence', 'there', 'thereafter', 'thereby']
stopwords += ['therefore', 'therein', 'thereupon', 'these', 'they']
stopwords += ['thick', 'thin', 'third', 'this', 'those', 'though', 'three']
stopwords += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stopwords += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stopwords += ['twenty', 'two', 'un', 'under', 'until', 'up', 'upon']
stopwords += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stopwords += ['whatever', 'when', 'whence', 'whenever', 'where']
stopwords += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stopwords += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stopwords += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stopwords += ['within', 'without', 'would', 'yet', 'you', 'your']
stopwords += ['yours', 'yourself', 'yourselves']
stopwords += ['january', 'february', 'march', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
# author name to remove
stopwords +=['karina nielsen','kevin daniels','elaine', 'jaewan yang','andrew O herdman','amanda', 'sabine','kerstin','kertin','sagepub','journalspermissions','catheine','chidiebere ogbonnaya','john wiley','sons ltd','martin','jeffrey B arthur']

In [12]:
data_proc = [filterWords(f, stopwords) for f in pdf_p] #running

In [13]:
data_proc[22]

'. author reprints permissions meta analysis interactive additive relative effects cognitive ability motivation performance tested longstanding belief performance function interaction cognitive ability motivation.using data values obtained primary study authors input used meta analysis assess strength consistency multiplicative effects ability motivation performance.triangulation evidence based types analyses revealed effects ability motivation performance additive multiplicative.additive effects ability motivation accounted explained variance performance ability motivation interaction accounted explained variance addition interaction consistently reflect predicted form findings acknowledgments article accepted editorship previous version paper presented meetings academy management thank brian anonymous reviewers useful constructive feedback allowed improve manuscript substantially thank guidance range restriction corrections.addition thank fred oswald help regression analysis.finally 

# Get country name

In [15]:
import pycountry

country_list = []
for country in pycountry.countries:
    a = (country.name).lower()
    country_list.append(a) 

In [17]:
data_proc_country =[]
for j in data_proc:
    line = j
    for k in country_list:
        line = re.sub(k, '', line)
        
    data_proc_country.append(line)

In [20]:
#data_proc_country

In [21]:
# save data
pickle.dump(data_proc_country, open( "C:/Thesis/Data/save/Master_Data/MD_3/data_proc_latest/JM_data_proc.p", "wb" ) )

In [22]:
#import io
with open("C:/Thesis/Data/save/Master_Data/MD_3/data_proc_latest/JM_data_proc_txt.txt", "w",encoding="utf-8") as outfile:
    for i in range(len(data_proc_country)):
        outstring = ""
        outstring += str(data_proc_country[i])
        outfile.write(outstring)